In [5]:
# ============================================
# Configuración inicial
# ============================================

# Librerías básicas
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Cargar variables de entorno (.env)
from dotenv import load_dotenv
load_dotenv()

# Comprobar entorno y versión
print("Entorno activo:", os.getcwd())
print("Versión de pandas:", pd.__version__)


Entorno activo: c:\Users\Diego Garcia\Documents\Cristian\DS-SoyHenry\Modulo 1\Código\notebooks
Versión de pandas: 2.3.3


In [7]:
# ============================================
# Configuración de rutas del proyecto
# ============================================

from pathlib import Path

# Directorios base del proyecto
NB_DIR = Path.cwd()                 # Carpeta donde está el notebook actual
PROJ_DIR = NB_DIR.parent            # Subimos un nivel: raíz del proyecto
DATA_RAW = PROJ_DIR / "data" / "raw"          # Datos originales
DATA_INTERIM = PROJ_DIR / "data" / "interim"  # Datos intermedios
DATA_PROCESSED = PROJ_DIR / "data" / "processed"  # Datos limpios finales

# Mostrar para verificar que las rutas son correctas
print("Proyecto:", PROJ_DIR)
print("Data RAW:", DATA_RAW)
list(DATA_RAW.iterdir())


Proyecto: c:\Users\Diego Garcia\Documents\Cristian\DS-SoyHenry\Modulo 1\Código
Data RAW: c:\Users\Diego Garcia\Documents\Cristian\DS-SoyHenry\Modulo 1\Código\data\raw


[WindowsPath('c:/Users/Diego Garcia/Documents/Cristian/DS-SoyHenry/Modulo 1/Código/data/raw/.gitkeep'),
 WindowsPath('c:/Users/Diego Garcia/Documents/Cristian/DS-SoyHenry/Modulo 1/Código/data/raw/base_datos_restaurantes_USA_v2.csv'),
 WindowsPath('c:/Users/Diego Garcia/Documents/Cristian/DS-SoyHenry/Modulo 1/Código/data/raw/yelp_restaurants.csv')]

In [11]:
# ============================================
# Función robusta para leer CSV
# ============================================

def read_csv_robust(path, **kwargs):
    """
    Intenta leer un archivo CSV probando distintas codificaciones (encoding)
    y separadores comunes. Evita errores típicos por datos con formato inusual.
    Adaptada para pandas >= 2.3 (sin low_memory).
    """
    encodings = ["utf-8", "latin-1", "cp1252"]   # codificaciones frecuentes
    seps = [",", ";", "\t", "|"]                 # separadores comunes
    last_err = None

    for enc in encodings:
        for sep in seps:
            try:
                df = pd.read_csv(
                    path, encoding=enc, sep=sep, engine="python",
                    on_bad_lines="skip", **kwargs   # 👈 eliminamos low_memory
                )
                # Si el separador fue incorrecto (solo una columna), reintenta
                if df.shape[1] == 1 and sep != ",":
                    continue
                print(f"✅ Archivo leído con encoding='{enc}', sep='{sep}' → columnas: {df.shape[1]}")
                return df
            except Exception as e:
                last_err = e
                continue
    raise RuntimeError(f"No se pudo leer {path} (último error: {last_err})")



In [12]:
# ============================================
# Cargar los archivos CSV
# ============================================

base_path = DATA_RAW / "base_datos_restaurantes_USA_v2.csv"
yelp_path = DATA_RAW / "yelp_restaurants.csv"

df_base = read_csv_robust(base_path)
df_yelp = read_csv_robust(yelp_path)

# Mostrar las dimensiones para confirmar carga
print("df_base:", df_base.shape)
print("df_yelp:", df_yelp.shape)


✅ Archivo leído con encoding='utf-8', sep=',' → columnas: 17
✅ Archivo leído con encoding='utf-8', sep=',' → columnas: 12
df_base: (30000, 17)
df_yelp: (463, 12)


In [13]:
# ============================================
# Inspección inicial del contenido
# ============================================

display(df_base.head(3))
display(df_yelp.head(3))

print("\nINFO df_base")
df_base.info()

print("\nINFO df_yelp")
df_yelp.info()

print("\nColumnas df_base:", list(df_base.columns))
print("Columnas df_yelp:", list(df_yelp.columns))


,id_persona,nombre,apellido,edad,genero,ciudad_residencia,estrato_socioeconomico,frecuencia_visita,promedio_gasto_comida,ocio,consume_licor,preferencias_alimenticias,membresia_premium,telefono_contacto,correo_electronico,tipo_de_pago_mas_usado,ingresos_mensuales
0,2550327378,Jackson,Gomez,31.0,Masculino,Miami,Alto,6,67.51,Sí,No,Vegetariano,Sí,(830)220-1926,NaN,Efectivo,6425
1,9446112038,Samantha,Soto,40.0,Femenino,Denver,Medio,2,44.92,Sí,Sí,Mariscos,No,881-476-1426,NaN,Efectivo,2374
2,3098363243,Terry,Adams,62.0,Femenino,Denver,Bajo,2,9.24,Sí,Sí,Vegetariano,No,NaN,diana74@example.net,Efectivo,1110


,alias,title,id,name,price,rating,review_count,distance,coordinates_latitude,coordinates_longitude,location_address1,city
0,asianfusion,Asian Fusion,K3ukx2e11xTRtYBU01dmrA,Salty Flame,No especificado,4.4,184,9568.050891,25.76022,-80.19267,1414 Brickell Ave,Miami
1,steak,Steakhouses,K3ukx2e11xTRtYBU01dmrA,Salty Flame,No especificado,4.4,184,9568.050891,25.76022,-80.19267,1414 Brickell Ave,Miami
2,cocktailbars,Cocktail Bars,K3ukx2e11xTRtYBU01dmrA,Salty Flame,No especificado,4.4,184,9568.050891,25.76022,-80.19267,1414 Brickell Ave,Miami



INFO df_base
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id_persona                 30000 non-null  int64  
 1   nombre                     30000 non-null  object 
 2   apellido                   30000 non-null  object 
 3   edad                       29899 non-null  float64
 4   genero                     30000 non-null  object 
 5   ciudad_residencia          30000 non-null  object 
 6   estrato_socioeconomico     30000 non-null  object 
 7   frecuencia_visita          30000 non-null  int64  
 8   promedio_gasto_comida      29855 non-null  float64
 9   ocio                       30000 non-null  object 
 10  consume_licor              30000 non-null  object 
 11  preferencias_alimenticias  28597 non-null  object 
 12  membresia_premium          30000 non-null  object 
 13  telefono_contacto          14834

In [15]:
# ============================================
# Análisis de calidad de datos
# ============================================

def resumen_calidad(df, nombre="df"):
    n = len(df)
    nul_pct = (df.isna().sum().sort_values(ascending=False) / n * 100).round(2)
    dup_rows = df.duplicated().sum()
    print(f"--- {nombre} ---")
    print(f"Filas: {n} | Columnas: {df.shape[1]} | Duplicados: {dup_rows}")
    display(nul_pct.to_frame("porcentaje_nulls_%").head(20))

resumen_calidad(df_base, "df_base")
resumen_calidad(df_yelp, "df_yelp")

# Estadísticas numéricas básicas
display(df_base.describe(include="number").T.head(10))
display(df_yelp.describe(include="number").T.head(10))


--- df_base ---
Filas: 30000 | Columnas: 17 | Duplicados: 0


,porcentaje_nulls_%
telefono_contacto,50.55
correo_electronico,50.24
preferencias_alimenticias,4.68
promedio_gasto_comida,0.48
edad,0.34
id_persona,0.00
genero,0.00
apellido,0.00
nombre,0.00
frecuencia_visita,0.00


--- df_yelp ---
Filas: 463 | Columnas: 12 | Duplicados: 0


,porcentaje_nulls_%
alias,0.0
title,0.0
id,0.0
name,0.0
price,0.0
rating,0.0
review_count,0.0
distance,0.0
coordinates_latitude,0.0
coordinates_longitude,0.0


,count,mean,std,min,25%,50%,75%,max
id_persona,30000.0,5.504765e+09,2.602799e+09,1.000153e+09,3.243617e+09,5.515865e+09,7.754426e+09,9.999627e+09
edad,29899.0,4.966501e+01,2.383955e+01,-5.000000e+00,3.300000e+01,4.900000e+01,6.500000e+01,3.000000e+02
frecuencia_visita,30000.0,3.896133e+00,2.741532e+00,-3.000000e+00,2.000000e+00,4.000000e+00,5.000000e+00,1.000000e+01
promedio_gasto_comida,29855.0,3.260345e+01,2.640260e+01,0.000000e+00,1.329000e+01,2.551000e+01,4.440000e+01,1.499700e+02
ingresos_mensuales,30000.0,5.389756e+03,4.538492e+03,8.000000e+02,1.860000e+03,3.402000e+03,7.761000e+03,1.799900e+04


,count,mean,std,min,25%,50%,75%,max
rating,463.0,4.342117,0.332863,2.700000,4.100000,4.400000,4.500000,5.000000
review_count,463.0,538.408207,852.570412,1.000000,87.500000,248.000000,686.000000,6869.000000
distance,463.0,6155.117432,3175.186957,132.202770,3175.275975,5518.960654,9375.791147,13539.614717
coordinates_latitude,463.0,25.764661,0.029770,25.682257,25.749597,25.765025,25.782240,25.842900
coordinates_longitude,463.0,-80.254168,0.048691,-80.393596,-80.281227,-80.257420,-80.210630,-80.185336
